In [6]:
import numpy as np
from dobbel import dobbellogger
from help_functies import *
from calibrator3 import calibrate
from matrix_helper import *
import matplotlib.pyplot as plt
import pandas as pd
import time
dob = dobbellogger()

In [19]:
def calibrate(dob, logtime, waittime, freq, acc_range, gyro_range):
    measurements = []
    Y_list = []
    for i, (column, sign) in zip(range(6), [(2, 1), (0, 1), (1, 1), (1, -1), (0, -1), (2, -1)]):
        print("Leg de dobbelsteen met nummer ", i + 1, " boven")
        print(f"Wacht {waittime} secondes")
        time.sleep(waittime)
        dob.connect()

        #Log the data of the dice
        dob.log(logtime, freq, acc_range, gyro_range)

        # Download the logged data from the dice
        dob.download()
        data = dob.datadf
        data = remove_nan(data)
        data_arr = np.array([data['x_acc'], data['y_acc'], data['z_acc']])
        measurements.append(data_arr.T)
        y_arr = np.zeros((data_arr.shape[1], 3))
        y_arr[:, column] = sign
        Y_list.append(y_arr)
    
    Y = np.concatenate(Y_list)
    
    w = np.concatenate(measurements)
    w = np.hstack((w, np.ones((w.shape[0], 1))))
    
    print(w.shape, Y.shape)
    
    X = np.matmul(np.matmul(np.linalg.inv(np.matmul(w.T, w)), w.T), Y)
    
    acc_bias = np.reshape(X[3, :], (3, 1))
    acc_rotmat = X[0:3, :].T
    
    cali = {
        'acc rotmat': acc_rotmat,
        'acc bias': acc_bias
    }

    return cali

In [20]:
cali = calibrate(dob, 3, 1, 100, 2, 125)

Leg de dobbelsteen met nummer  1  boven
Wacht 1 secondes
Logging data for 3s
Done! The data is located in self.datadf
Leg de dobbelsteen met nummer  2  boven
Wacht 1 secondes
Logging data for 3s
Done! The data is located in self.datadf
Leg de dobbelsteen met nummer  3  boven
Wacht 1 secondes
Logging data for 3s
Done! The data is located in self.datadf
Leg de dobbelsteen met nummer  4  boven
Wacht 1 secondes
Logging data for 3s
Done! The data is located in self.datadf
Leg de dobbelsteen met nummer  5  boven
Wacht 1 secondes
Logging data for 3s
Done! The data is located in self.datadf
Leg de dobbelsteen met nummer  6  boven
Wacht 1 secondes
Logging data for 3s
Done! The data is located in self.datadf
(1840, 4) (1840, 3)


In [9]:
print(np.concatenate([np.array([[1,2],[3,4]]), np.array([[1,2],[3,4],[5,6]])]))

[[1 2]
 [3 4]
 [1 2]
 [3 4]
 [5 6]]


In [22]:
print(cali['acc rotmat'])
print(cali['acc bias'])

[[-0.02536098  0.99892451  0.00277794]
 [ 0.77247654  0.02525039 -0.63044882]
 [-0.62355458 -0.01429244 -0.75850794]]
[ 0.0153939   0.00340093 -0.01406814]


In [26]:
dob.connect()
dob.log(5, 100, 4, 2000)
dob.download()
data = dob.datadf

Logging data for 5s
Done! The data is located in self.datadf


In [27]:
data = remove_nan(data)
for column in data.columns:
    print(column, np.mean(data[column]))

timestamp 2490.2848605577688
x_acc -0.6494816079255422
y_acc -0.048303558670148954
z_acc -0.8202234010513344
x_gyro -0.1532727220931894
y_gyro 0.4462615277641107
z_gyro 0.04379267955400598


In [28]:
data.head(1000)

,timestamp,x_acc,y_acc,z_acc,x_gyro,y_gyro,z_gyro
0,0.0,-0.649048,-0.047852,-0.818848,-0.365854,0.731707,0.365854
1,61.0,-0.648315,-0.047241,-0.822510,-0.304878,0.731707,-0.426829
2,72.0,-0.648682,-0.046143,-0.822754,-0.121951,0.304878,0.121951
3,81.0,-0.649048,-0.047852,-0.820435,-0.121951,0.548781,0.000000
4,91.0,-0.651123,-0.049561,-0.821411,-0.121951,0.426829,0.060976
...,...,...,...,...,...,...,...
497,4888.0,-0.650269,-0.048584,-0.820923,-0.182927,0.487805,0.060976
498,4898.0,-0.650513,-0.049561,-0.818115,-0.060976,0.426829,0.000000
499,4909.0,-0.648071,-0.050171,-0.818970,-0.182927,0.548781,0.121951
500,4917.0,-0.650757,-0.050537,-0.820923,-0.060976,0.487805,0.000000


In [29]:
data['x_acc'] = data['x_acc'] - cali['acc bias'][0]
data['y_acc'] = data['y_acc'] - cali['acc bias'][1]
data['z_acc'] = data['z_acc'] - cali['acc bias'][2]

In [30]:
data.head(1000)

,timestamp,x_acc,y_acc,z_acc,x_gyro,y_gyro,z_gyro
0,0.0,-0.645060,-0.034433,-0.813114,-0.365854,0.731707,0.365854
1,61.0,-0.644328,-0.033823,-0.816777,-0.304878,0.731707,-0.426829
2,72.0,-0.644694,-0.032724,-0.817021,-0.121951,0.304878,0.121951
3,81.0,-0.645060,-0.034433,-0.814701,-0.121951,0.548781,0.000000
4,91.0,-0.647136,-0.036142,-0.815678,-0.121951,0.426829,0.060976
...,...,...,...,...,...,...,...
497,4888.0,-0.646281,-0.035165,-0.815190,-0.182927,0.487805,0.060976
498,4898.0,-0.646525,-0.036142,-0.812382,-0.060976,0.426829,0.000000
499,4909.0,-0.644084,-0.036752,-0.813236,-0.182927,0.548781,0.121951
500,4917.0,-0.646769,-0.037119,-0.815190,-0.060976,0.487805,0.000000


In [52]:
dob.connect()
dob.log(3, 100, 2, 125)
dob.download()
data = dob.datadf
data = remove_nan(data)
data['x_acc'] = data['x_acc'] - cali['acc bias'][0]
data['y_acc'] = data['y_acc'] - cali['acc bias'][1]
data['z_acc'] = data['z_acc'] - cali['acc bias'][2]
y_mean_1 = np.mean(np.array([data['x_acc'], data['y_acc'], data['z_acc']]), axis=1)

Logging data for 3s
Done! The data is located in self.datadf


In [53]:
dob.connect()
dob.log(3, 100, 2, 125)
dob.download()
data = dob.datadf
data = remove_nan(data)
data['x_acc'] = data['x_acc'] - cali['acc bias'][0]
data['y_acc'] = data['y_acc'] - cali['acc bias'][1]
data['z_acc'] = data['z_acc'] - cali['acc bias'][2]
y_mean_2 = np.mean(np.array([data['x_acc'], data['y_acc'], data['z_acc']]), axis=1)

Logging data for 10s
Done! The data is located in self.datadf


In [54]:
gb = y_mean_1 / np.linalg.norm(y_mean_1)
gn = np.array([0, 0, 1])
mb = np.cross(gb, np.cross(y_mean_2 / np.linalg.norm(y_mean_2), gb))
mn = np.array([1, 0, 0])

A = - np.matmul(left_quat_mul(np.array([0, *gn])), right_quat_mul(np.array([0, *gb]))) - np.matmul(left_quat_mul(np.array([0, *mn])), right_quat_mul(np.array([0, *mb])))
eigenvalues, eigenvectors = np.linalg.eigh(A)
max_eigenvalue_index = np.argmax(eigenvalues)
max_eigenvector = eigenvectors[:, max_eigenvalue_index]
q1 = max_eigenvector

In [55]:
print(q1)

[-0.22690776 -0.66348662 -0.67259935  0.23644974]


In [56]:
dob.connect()
dob.log(5, 100, 4, 500)
dob.download()
data = dob.datadf

Logging data for 5s
Done! The data is located in self.datadf


In [57]:
data['x_acc'] = data['x_acc'] - cali['acc bias'][0]
data['y_acc'] = data['y_acc'] - cali['acc bias'][1]
data['z_acc'] = data['z_acc'] - cali['acc bias'][2]

In [58]:
rot_data = rotate_data(data, q1)

In [59]:
rot_data = remove_nan(rot_data)
for column in rot_data.columns:
    print(column, np.mean(rot_data[column]))

timestamp 2466.753968253968
x_acc 0.9851213054082968
y_acc -0.012294273545748042
z_acc 0.017233311226701728
x_gyro 0.45262506757465343
y_gyro -0.12986940928525784
z_gyro 0.04092281887050569
